In [3]:
import os, random, numpy as np, torch, evaluate
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed
)
from peft import LoraConfig, get_peft_model
from transformers_interpret import SequenceClassificationExplainer

In [8]:
MODEL_CHECKPOINT = "distilbert-base-uncased"
LABEL2ID = {"negative": 0, "positive": 1}
ID2LABEL = {0: 'Negative', 1: 'Positive'}
MAX_LEN = 512
RANDOM_STATE = 42
BATCH_SIZE = 16
BASE_LR = 2e-4
NUM_EPOCHS = 3
WARMUP_RATIO = 0.06

set_seed(RANDOM_STATE)

In [11]:
raw_ds = load_dataset("nocode-ai/imdb-movie-reviews", split="train")
texts = raw_ds["review"]
labels = []
for x in raw_ds['sentiment']:
    labels.append(LABEL2ID[x])

In [21]:
raw_ds

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

In [16]:
# 80 - 10 - 10 (train - test - split)
tr_texts, temp_texts, tr_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=RANDOM_STATE
)

val_texts, te_texts, val_labels, te_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=RANDOM_STATE
)

In [17]:
def make_hf_dataset(texts, labels):
    return {"review": texts, "label": labels}

In [19]:
dataset = DatasetDict({
    "train": raw_ds.from_dict(make_hf_dataset(tr_texts,  tr_labels)),
    "validation": raw_ds.from_dict(make_hf_dataset(val_texts, val_labels)),
    "test": raw_ds.from_dict(make_hf_dataset(te_texts,  te_labels)),
})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 5000
    })
})

In [22]:
tok = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, add_prefix_space = True)
if tok.pad_token is None:
    tok.add_special_token({'pad_token': '[PAD]'})

In [23]:
def tokenize(batch):
    return tok(
        batch['review'],
        padding='max_length',
        truncation=True,
        max_length=MAX_LEN
    )

In [24]:
tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=['review'])

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [33]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [34]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(LABEL2ID),
    id2label=ID2LABEL,
    label2id=LABEL2ID,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
TARGET_MODULES = ["q_lin", "k_lin", "v_lin", "out_lin"]
lora_cfg = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=TARGET_MODULES,
)
model = get_peft_model(base_model, lora_cfg)
model.resize_token_embeddings(len(tok))
model.print_trainable_parameters()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [36]:
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

In [37]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1":       metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

In [39]:
args = TrainingArguments(
    output_dir="distilbert-imdb-lora",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=BASE_LR,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    num_train_epochs=NUM_EPOCHS,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    weight_decay=0.01,
    gradient_checkpointing=True,
    fp16=torch.cuda.is_available(),  # halves memory if on GPU with FP16 support
    report_to="none",
    seed=RANDOM_STATE,
    dataloader_pin_memory=True,
)

In [40]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tok,
    data_collator=DataCollatorWithPadding(tok),
    compute_metrics=compute_metrics,
)


/var/folders/76/b119mp7x4fs63_bpw5mg9tc00000gn/T/ipykernel_10696/1939329306.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [41]:
trainer.train()

/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.355100,0.315118,0.870800,0.870728
2,0.347100,0.302534,0.873200,0.873198
3,0.322600,0.300383,0.873200,0.873190


/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/AshishR_T/miniconda3/envs/ai4all_py310/lib/python3.10/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=7500, training_loss=0.35928014526367186, metrics={'train_runtime': 4240.5746, 'train_samples_per_second': 28.298, 'train_steps_per_second': 1.769, 'total_flos': 1.62230870016e+16, 'train_loss': 0.35928014526367186, 'epoch': 3.0})

In [42]:
explainer = SequenceClassificationExplainer(
    model=trainer.model,
    tokenizer=tok,
    attribution_type="lig",  # LayerIntegratedGradients
)

In [43]:
sample_text = (
    "I expected a mediocre rom-com, but it turned out to be a heartfelt story "
    "with brilliant performances and sharp dialogue."
)
word_attributions = explainer(sample_text)

In [46]:
print(f"\nPrediction: {explainer.predicted_class_name}\nConfidence: {explainer.predicted_class_index:.4f}")
print("Top explanatory tokens:")
for word, score in word_attributions[:]:
    print(f"{word:>12s}  {score: .3f}")


Prediction: Positive
Confidence: 1.0000
Top explanatory tokens:
       [CLS]   0.000
           i  -0.154
    expected  -0.392
           a   0.011
         med   0.197
        ##io   0.056
       ##cre  -0.087
         rom  -0.174
           -   0.088
         com  -0.189
           ,  -0.175
         but  -0.060
          it  -0.073
      turned  -0.132
         out  -0.135
          to   0.058
          be  -0.072
           a  -0.057
       heart   0.040
      ##felt   0.130
       story  -0.278
        with   0.072
   brilliant   0.375
performances   0.045
         and   0.150
       sharp   0.443
    dialogue  -0.263
           .  -0.283
       [SEP]   0.000


In [47]:
explainer.visualize("distilbert_viz.html")